In [28]:
import time
import json
import random
import numpy as np
from glob import glob
import matplotlib as mpl
from copy import deepcopy
from smol.io import load_work
from smol.moca import Sampler
from smol.moca import Ensemble
from smol.cofe.space import Vacancy
from smol.moca.sampler.mcusher import Tableflip
from pymatgen.core.sites import Species
from pymatgen.core.structure import Structure
from pymatgen.transformations.standard_transformations \
import (OxidationStateDecorationTransformation, \
        OrderDisorderedStructureTransformation)
from monty.serialization import dumpfn, loadfn
from smol.moca.sampler.container import SampleContainer
from matplotlib import pyplot as plt

## 0. Functions

In [29]:
# For sanity check

def get_dim_ids_by_sublattice(bits):
    """Get the component index of each species in vector n.

    Args:
        bits(List[List[Specie|Vacancy|Element]]):
           Species on each sub-lattice.
    Returns:
        Component index of each species on each sublattice in vector n:
           List[List[int]]
    """
    dim_ids = []
    dim_id = 0
    for species in bits:
        dim_ids.append(list(range(dim_id, dim_id + len(species))))
        dim_id += len(species)
    return dim_ids


def flip_vec_to_reaction(u, bits):
    """Convert flip direction into a reaction formula in string.

    This function is for easy interpretation of flip directions.
    Args:
        u(1D ArrayLike[int]):
            The flip vector in number change of species.
        bits(List[List[Specie|DummySpecie|Element|Vacancy]]):
            Species on all sub-lattices.
    Return:
        Reaction formulas: str.
    """
    u = np.array(u, dtype=int)
    dim_ids = get_dim_ids_by_sublattice(bits)

    from_strs = []
    to_strs = []

    for sl_id, (sl_species, sl_dims) in enumerate(zip(bits, dim_ids)):
        for specie, dim in zip(sl_species, sl_dims):
            if u[dim] < 0:
                from_strs.append('{} {}({})'
                                 .format(-u[dim], str(specie), sl_id))
            elif u[dim] > 0:
                to_strs.append('{} {}({})'
                               .format(u[dim], str(specie), sl_id))

    from_str = ' + '.join(from_strs)
    to_str = ' + '.join(to_strs)
    return from_str + ' -> ' + to_str


def set_rc_params():
    """
    General params for plot.
    """
    params = {'axes.linewidth': 1.5,
              'axes.unicode_minus': False,
              'figure.dpi': 300,
              'font.size': 25,
              'legend.frameon': False,
              'legend.handletextpad': 0.4,
              'legend.handlelength': 1,
              'legend.fontsize': 10,
              'lines.markeredgewidth': 4,
              'lines.linewidth': 3,
              'lines.markersize': 15,
              'mathtext.default': 'regular',
              'savefig.bbox': 'tight',
              'xtick.labelsize': 20,
              'ytick.labelsize': 20,
              'xtick.major.size': 5,
              'xtick.minor.size': 3,
              'ytick.major.size': 5,
              'ytick.minor.size': 3,
              'xtick.major.width': 1,
              'xtick.minor.width': 0.5,
              'ytick.major.width': 1,
              'ytick.minor.width': 0.5,
              'xtick.top': True,
              'ytick.right': True,
              'axes.edgecolor': 'black',
              'legend.fancybox': True,
              'figure.figsize': [8, 6]}
    for p in params:
        mpl.rcParams[p] = params[p]

In [27]:
# Trajectory retriever
# Finished functions

def read_json(fjson):
    with open(fjson) as f:
        return json.load(f)


def write_json(d, fjson):
    with open(fjson, 'w') as f:
        json.dump(d, f)
    return d


def get_data_from_hdf5(saved_path):
    
    return


def get_data_from_nps(saved_path):
    
    return


def get_discard_length():
    
    # Need low temperature run data to test
    
    return


def get_sampling_efficiency(energies, discard_length, sampling_length):
    """
    Args:
        energies: NumPy array. Sampled energy.
        discard_length: Int. Equilibrium length of the sample.
        sampling_length: Int. Sampling length to get sampling efficiency.
    Returns:
        tot_var: total variance of the entire sample.
        block_var: block variance.
        sampling_efficiency: Idealy 1.
    """
    blocks = []
    block_means = []
    start_position = discard_length
    tot_var = np.var(energies[discard_length:])

    while start_position + sampling_length < len(energies):
        blocks.append(energies[start_position : start_position + sampling_length])
        start_position += sampling_length

    for i in blocks:
        block_means.append(i.mean())
    block_var = np.var(block_means)
    sampling_efficiency = tot_var / sampling_length / block_var
    
    return tot_var, block_var, sampling_efficiency


def get_thin_trajectory(species_count, discard_length, thin_by):
    """
    Args:
        species_count: Dict. of counted number of species
        discard_length: Int. Equilibrium length of the sample.
        thin_by: Int. Thin by number.
    Returns:
        thin_trajectory: List of tuple (Ca2+, Na+). Thinned tracjectory
        average: Average value of ensemble.
    """
    # Assume that sampled from 5x5x6 size ensemble
    thin_trajectory = []
    discarded_na = species_count['Na+'][discard_length:]
    discarded_ca = species_count['Ca2+'][discard_length:]
    ca_total = 0
    na_total = 0
    count = 0
    for i, j in enumerate(discarded_na):
        if i % thin_by == 0:
            thin_trajectory.append((discarded_ca[i]/300, j/300))
        count += 1
        ca_total += discarded_ca[i]/300
        na_total += j/300
    ca_average = ca_total / count
    na_average = na_total / count
    
    return thin_trajectory, (ca_average, na_average)


def get_all_trajectories(saved_dir, discard_length, thin_by):
    
    traj_data = {}
    species_count_list = []
    saved_list = glob(saved_dir + "/*")
    for i in saved_list:
        if "species_count" in i:
            species_count_list.append(i)
    for j in species_count_list:
        spec_species_count = read_json(j)
        spec_species_count = json.loads(spec_species_count)
        thin_trajectory, average = get_thin_trajectory(spec_species_count, discard_length, thin_by)
        # Need to automate key generation
        key = '_'.join(j.split("/")[-1].split("_")[:2])
        traj_data[key] = {}
        traj_data[key]['trajectory'] = thin_trajectory
        traj_data[key]['average'] = average
    
    return traj_data

## 1. Load data from cluster expansion fitting & Generate Ensemble
Note: For the details of fitting, refer to groupCE_CaNaVP

In [5]:
work = load_work('/global/scratch/users/yychoi94/CaNaVP/data/final_data/ce/final_canvp_ce.mson')
expansion = work['ClusterExpansion']
wrangler = work['StructureWrangler']
print("{} Non-zero ECIS over {} cluster ECIS".format(np.count_nonzero(expansion.coefs), len(expansion.coefs)))

103 Non-zero ECIS over 629 cluster ECIS


## 2. Set Basic variables. 

In [6]:
# Atoms.
na = Species('Na', 1)
ca = Species('Ca', 2)
v3 = Species('V', 3)
v4 = Species('V', 4)
v5 = Species('V', 5)
vac = Vacancy()

# Supercell matrix
sc_matrix = np.array([[5, 0, 0],
                      [0, 5, 0],
                      [0, 0, 6]])

# Relative chemical potentials
chemical_potentials = {'Na+':0, 'Ca2+': 0, 'Vacancy': 0,
                       'V3+': 0, 'V4+': 0, 'V5+': 0}

## 3. Load ensemble.

In [8]:
ensemble_path = '/global/home/users/yychoi94/notebooks/final_canvp_ensemble_1201.mson'
ensemble = loadfn(ensemble_path)
# dumpfn(ensemble, ensemble_path)

In [9]:
for sublattice in ensemble.sublattices:
    print(sublattice.species)

(Species O2-,)
(Species Na+, Species Ca2+, Vacancy vacA0+)
(Species V3+, Species V4+, Species V5+)
(Species P5+,)


In [10]:
# Specific flip table based on saved ensemble.
flip_table = np.array([[0,  -1,  0,  1,  -1,  1,  0,  0],
                       [0,  -1,  0,  1,  0,  -1,  1,  0],
                       [0,  0,  -1,  1,  -1,  0,  1,  0],
                       [0,  -2,  1,  1,  0,  0,  0,  0]])

In [11]:
start = time.time()
sampler = Sampler.from_ensemble(ensemble, step_type="tableflip",
                                temperature=2000, optimize_basis=False, flip_table=flip_table)

print(f"Sampling information: {sampler.samples.metadata}")
end = time.time()
print(f"{end-start}s for initialization.")

/global/home/users/yychoi94/codes/smol/smol/moca/sampler/mcusher.py:358: UserWarning: Current occupancy violates compspace constraints! Are you initializing trace?
  warnings.warn("Current occupancy violates compspace constraints! "


Sampling information: {'kernel': 'Metropolis', 'step': 'tableflip', 'chemical_potentials': {Species Na+: 0, Species Ca2+: 0, Vacancy vacA0+: 0, Species V3+: 0, Species V4+: 0, Species V5+: 0}, 'seeds': [156778177983409017164721517047867786276]}
0.798330545425415s for initialization.


## 4. Retrieve trajectory

In [30]:
set_rc_params()

In [21]:
save_dir = "/global/scratch/users/yychoi94/CaNaVP_gcMC/300K_556_30_6020/data"
saved_list = glob(save_dir + "/*")
tot_cation_site = round(np.linalg.det(ensemble.processor.supercell_matrix)) * 8

In [22]:
test_traj_data = get_all_trajectories(save_dir, 50000, 5000)

In [45]:
def get_vc_data(traj_data, cation='Na'):
    
    # Option: 
    voltages, cations = [], []
    for key in traj_data:
        if cation == 'Na':
            chempo = float(key.split('_')[-1])
            voltage = (-1.3122 - chempo)
            cations.append(test_traj_data[key]['average'][1])
        elif cation == 'Ca':
            chempo = float(key.split('_')[0])
            voltage = (-1.9985 - chempo) / 2
            cations.append(test_traj_data[key]['average'][0])
        voltages.append(voltage)
    sorted_voltages = [x for _, x in sorted(zip(cations, voltages))]
    cations.sort()
    
    return cations, sorted_voltages

In [46]:
cations, sorted_voltages = get_vc_data(test_traj_data, cation='Na')

In [74]:
for i, j in enumerate(cations):
    if j == 1.0:
        print(i)

33
34
35
36
37
38


In [75]:
- sorted_voltages[38] -1.3122

-4.72

In [73]:
- sorted_voltages[99] -1.3122

-3.2

## Retrieve Structure

In [81]:
occu_dir = '/global/scratch/users/yychoi94/CaNaVP_gcMC/300K_556_30_6020/data/-30.0_-4.72_occupancy.npz'

In [82]:
occupancies = np.load(occu_dir)['o']

In [83]:
final_structure = ensemble.processor.structure_from_occupancy(occupancies[-1])

In [84]:
final_structure.sort()

In [85]:
final_structure.to('POSCAR', './300_Na1_POSCAR')
np.save('./300_Na1_occu.npy', occupancies[-1])